In [1]:
cd ../../..

/home/cridonoso/Documents/astromer-code


# Using ASTROMER as layer's model

In [2]:
import tensorflow as tf
import pandas as pd
import os

from tensorflow.keras.layers import BatchNormalization, Dense, LSTM, LayerNormalization
from tensorflow.keras import Input, Model

from presentation.experiments.clf.classifiers import build_mlp_att, build_lstm_att
from core.data import load_dataset, inference_pipeline, pretraining_pipeline
from core.astromer import ASTROMER
from core.training.scheduler import CustomSchedule

%load_ext autoreload
%autoreload 2

2022-04-02 15:21:12.783877: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-02 15:21:12.783896: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


First we load ASTROMER pre-trained weights

In [3]:
astromer = ASTROMER()
astromer.load_weights('./runs/macho/')

2022-04-02 15:08:15.075368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-02 15:08:15.075409: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-02 15:08:15.075426: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (appa): /proc/driver/nvidia/version does not exist
2022-04-02 15:08:15.075812: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


First we load data

In [4]:
batch_size = 256
max_obs    = 200
datapath   = './data/records/alcock/fold_0/alcock/'

In [5]:
catalog = pd.read_csv('{}/objects.csv'.format(datapath))
n_classes = catalog.shape[0]
catalog

,label,size
0,Cep_0,1082
1,Cep_1,583
2,EC,6724
3,LPV,2946
4,RRab,7297
5,RRc,1662


In [6]:
traindata = load_dataset(os.path.join(datapath, 'train'), repeat=1, shuffle=True)
valdata   = load_dataset(os.path.join(datapath, 'val'), repeat=1)
testdata  = load_dataset(os.path.join(datapath, 'test'), repeat=1)

2022-04-02 15:21:21.259668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-02 15:21:21.259726: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-02 15:21:21.259740: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (appa): /proc/driver/nvidia/version does not exist
2022-04-02 15:21:21.259950: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[INFO] Shuffling


To evaluate use `pretraining_pipeline()` 

In [7]:
test_batches = pretraining_pipeline(testdata, batch_size=batch_size, max_obs=max_obs, 
                                    msk_frac=0.5, rnd_frac=0.2, same_frac= 0.2)

[INFO] Pretraining mode. Random 200-len windows


Before evaluating ASTROMER we need to compile the model. The `.compile()` function automatically loads the custom losses used during pre-training 

In [8]:
astromer.compile(optimizer='adam')

In [9]:
astromer.evaluate(test_batches.take(2))

2/2 [==============================] - 3s 1s/step - loss: 0.1686 - r2: 0.6605


[0.1841157078742981, 0.685067892074585]

## Classification

When using `inference_pipeline()` the output is the classification labels instead of the reconstruction

In [15]:
train_batches = inference_pipeline(traindata,
                                   batch_size=batch_size,
                                   max_obs=max_obs,
                                   n_classes=n_classes,
                                   shuffle=True)
val_batches   = inference_pipeline(valdata,
                                   batch_size=batch_size,
                                   max_obs=max_obs,
                                   n_classes=n_classes,
                                   shuffle=True)

[INFO] Inference mode. Cutting 200-len windows
[INFO] Inference mode. Cutting 200-len windows


In [16]:
for x, y  in train_batches:
    print(y)
    
    break

InvalidArgumentError: Expected begin and size arguments to be 1-D tensors of size 0, but got shapes [1] and [1] instead.
	 [[{{function_node __inference_get_masked_387}}{{node get_masked/Slice_1}}]] [Op:IteratorGetNext]

Now we use the encoder layer from the pretrained ASTROMER instance

In [25]:
astromer = ASTROMER()
astromer.load_weights('./runs/macho/')
encoder = astromer.get_layer('encoder')

In [27]:
serie  = Input(shape=(max_obs, 1), batch_size=None, name='input')
times  = Input(shape=(max_obs, 1), batch_size=None, name='times')
mask   = Input(shape=(max_obs, 1), batch_size=None, name='mask')
placeholder = {'input':serie, 'mask_in':mask, 'times':times}

mask_rnn = tf.cast(1.-placeholder['mask_in'][...,0], dtype=tf.bool)

x = encoder(placeholder, training=False)
x = LSTM(256, dropout=.3, return_sequences=True)(x, mask=mask_rnn)
x = LayerNormalization()(x)
x = LSTM(256, dropout=.3)(x, mask=mask_rnn)
x = LayerNormalization()(x)
x = Dense(n_classes)(x)

classifier = Model(inputs=placeholder, outputs=x, name="LSTMATT")

In [28]:
classifier.summary()

Model: "LSTMATT"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 200, 1)]     0           []                               
                                                                                                  
 tf.__operators__.getitem_4 (Sl  (None, 200)         0           ['mask[0][0]']                   
 icingOpLambda)                                                                                   
                                                                                                  
 input (InputLayer)             [(None, 200, 1)]     0           []                               
                                                                                                  
 times (InputLayer)             [(None, 200, 1)]     0           []                         

In [29]:
from tensorflow.keras.losses import CategoricalCrossentropy

classifier.compile(optimizer='adam',
                   loss=CategoricalCrossentropy(from_logits=True),
                   metrics='accuracy')

we use early stopping callback on our classifier

In [30]:
from tensorflow.keras.callbacks import EarlyStopping
estop = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=40,
                      verbose=0,
                      mode='auto',
                      baseline=None,
                      restore_best_weights=True)

In [31]:
history = classifier.fit(train_batches,
                         epochs=10000,
                         callbacks=[estop],
                         validation_data=val_batches)

Epoch 1/10000


InvalidArgumentError: Graph execution error:

Expected begin and size arguments to be 1-D tensors of size 0, but got shapes [1] and [1] instead.
	 [[{{node get_masked/Slice_1}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_68834]